# Using vehicle filters in Fleet Planner API

This jupyter notebook will run you through an example of using vehicle filters in the Fleet Planner API. We are going to get a recommendation plan for a fleet of vehicles while:

1. Excluding specific vehicles from performing a task
2. Allowing only specific vehicles to perform a task

In [ ]:
import requests
import os
import json
import copy

## API Key

To run this example, you'll need a rideOS API key. You can sign up for one [here](https://app.rideos.ai/login#lockScreen=signUp) and view it on your [profile page](https://app.rideos.ai/profile), then assign it to the `API_KEY` variable directly or set it as an environment variable to `RIDEOS_API_KEY`:

In [ ]:
# Load the rideOS API key
API_KEY = os.getenv("RIDEOS_API_KEY")
rideos_auth_header = {"X-Api-Key": API_KEY}

get_plan_url = 'https://api.rideos.ai/fleet/v2/GetPlan'

## Case 1: Exclude specific vehicles from performing a task

Specific vehicles can be excluded from performing a task. The corresponding task will never be assigned to such vehicles by the Fleet Planner. In this example, we have 2 vehicles `vehicle-0` and `vehicle-1` and 1 task `task-0`. `vehicle-0` has been excluded from performing the task. 

Even though `vehicle-0` is much closer, `vehicle-1` has been assigned to the task since `vehicle-0` has been excluded.

In [ ]:
exclude_pool_ride_request = {
    "optimizeFor": "RIDE_HAIL",
    "vehicles": {
        "vehicle-0": {
            "resourceCapacity": 4,
            "vehicleId": "vehicle-0",
            "position": {
                "latitude": 37.788897,
                "longitude": -122.389669
            }
        },
        "vehicle-1": {
            "resourceCapacity": 3,
            "vehicleId": "vehicle-1",
            "position": {
                "latitude": 37.773611,
                "longitude": -122.421216
            }
        }
    },
    "tasks": {
        "task-0": {
            "resourcesRequired": 1,
            "pickupStep": {
                "position": {
                    "latitude": 37.790329,
                    "longitude": -122.392231
                }
            },
            "dropoffStep": {
                "position": {
                    "latitude": 37.795107,
                    "longitude": -122.393426
                }
            },
            "blacklistedVehicleId" : ['vehicle-0']
        }
    }
}


## Recommendation plan for each vehicle

In the response, we will get the recommendation plan for each vehicle

In [ ]:
response = requests.post(get_plan_url, headers=rideos_auth_header, json=exclude_pool_ride_request)
response_dict = json.loads(response.text)
for recommendation in response_dict['recommendations']:
    steps = recommendation['planRecommendation']['assignedSteps']
    for step in steps:
        step['vehicle'] = recommendation['vehicleId']
        print(step)

## Case 2: Allow only specific vehicles to perform a task

The corresponding task will only be assigned to specific vehicles that have been allowed. `vehicle-1` has been allowed for `task-0`.

Even though `vehicle-0` is much closer to `task-0`, the Fleet Planner will assign `vehicle-1` to `task-0` as only that vehicle has been allowed for the task.

In [ ]:
allow_pool_ride_request = {
    "optimizeFor": "RIDE_HAIL",
    "vehicles": {
        "vehicle-0": {
            "resourceCapacity": 4,
            "vehicleId": "vehicle-0",
            "position": {
                "latitude": 37.788897,
                "longitude": -122.389669
            }
        },
        "vehicle-1": {
            "resourceCapacity": 3,
            "vehicleId": "vehicle-1",
            "position": {
                "latitude": 37.773611,
                "longitude": -122.421216
            }
        }
    },
    "tasks": {
        "task-0": {
            "resourcesRequired": 1,
            "pickupStep": {
                "position": {
                    "latitude": 37.790329,
                    "longitude": -122.392231
                }
            },
            "dropoffStep": {
                "position": {
                    "latitude": 37.795107,
                    "longitude": -122.393426
                }
            },
            "whitelistedVehicleId" : ['vehicle-1']
        }
    }
}


## Recommendation plan for each vehicle

In the response, we will get the recommendation plan for each vehicle

In [ ]:
response = requests.post(get_plan_url, headers=rideos_auth_header, json=allow_pool_ride_request)
response_dict = json.loads(response.text)
for recommendation in response_dict['recommendations']:
    steps = recommendation['planRecommendation']['assignedSteps']
    for step in steps:
        step['vehicle'] = recommendation['vehicleId']
        print(step)